# Community Notes Data

- We first filter the community notes data with only helpful and not helpful notes.
- We then select notes that are in English.
- Finally, we convert the data to a KTO trainer format as required by KTO Trainer, which can be seen from the following link: https://huggingface.co/docs/trl/main/en/kto_trainer

In [ ]:
# Filter the helpful notes from X's community notes data
## The data we downloaded includes contributions up until 1:32 AM · Mar 10, 2024
### first download the data from this link: https://twitter.com/i/communitynotes/download-data

import pandas as pd
import numpy as np

# Read the notes file
notes = pd.read_csv('/Users/barisyazici/Desktop/Community Notes Data/notes-00000.tsv', delimiter='\t')
notes_status = pd.read_csv('/Users/barisyazici/Desktop/Community Notes Data/noteStatusHistory-00000.tsv', delimiter='\t')

# Value counts
notes_status["currentStatus"].value_counts()

In [ ]:
# Collect helpful and not helpful note ids in seperate lists
helpful_note_ids = notes_status.loc[notes_status["currentStatus"] == "CURRENTLY_RATED_HELPFUL", "noteId"].to_list()
not_helpful_note_ids = notes_status.loc[notes_status["currentStatus"] == "CURRENTLY_RATED_NOT_HELPFUL", "noteId"].to_list()

# Filter the original notes data based on note ids
helpful_notes = notes[notes["noteId"].isin(helpful_note_ids)]
not_helpful_notes = notes[notes["noteId"].isin(not_helpful_note_ids)]

In [ ]:
# Helpful Notes

df_helpful_notes = helpful_notes

# Create the "prompt" column with default value
df_helpful_notes['prompt'] = ""

# Function to insert spaces before underscores
def insert_spaces_underscore(column):
    return ' '.join(word.capitalize() for word in column.split('_'))

# Function to insert spaces before capital letters
def insert_spaces_capital(column):
    return ''.join([' ' + c if c.isupper() else c for c in column]).strip()

# Iterate over each row
for index, row in df_helpful_notes.iterrows():
    # Adding spaces to the classification column
    classification = insert_spaces_underscore(row['classification'])
    misleading_columns = ['misleadingOther', 'misleadingFactualError',
                          'misleadingManipulatedMedia', 'misleadingOutdatedInformation',
                          'misleadingMissingImportantContext', 'misleadingUnverifiedClaimAsFact',
                          'misleadingSatire', 'notMisleadingOther',
                          'notMisleadingFactuallyCorrect',
                          'notMisleadingOutdatedButNotWhenWritten', 'notMisleadingClearlySatire',
                          'notMisleadingPersonalOpinion']
    
    # Find the column name with value 1
    for column in misleading_columns:
        if row[column] == 1:
            misleading_column = insert_spaces_capital(column)
            break

    # Fill in the prompt column
    prompt_text = f"Please explain the evidence behind your choice of labeling this tweet as {classification}, to help others who see this tweet understand why it is {misleading_column}"
    df_helpful_notes.at[index, 'prompt'] = prompt_text

In [ ]:
# Unhelpful Notes

df_not_helpful_notes = not_helpful_notes

# Create the "prompt" column with default value
df_not_helpful_notes['prompt'] = ""

# Function to insert spaces before underscores
def insert_spaces_underscore(column):
    return ' '.join(word.capitalize() for word in column.split('_'))

# Function to insert spaces before capital letters
def insert_spaces_capital(column):
    return ''.join([' ' + c if c.isupper() else c for c in column]).strip()

# Iterate over each row
for index, row in df_not_helpful_notes.iterrows():
    # Adding spaces to the classification column
    classification = insert_spaces_underscore(row['classification'])
    misleading_columns = ['misleadingOther', 'misleadingFactualError',
                          'misleadingManipulatedMedia', 'misleadingOutdatedInformation',
                          'misleadingMissingImportantContext', 'misleadingUnverifiedClaimAsFact',
                          'misleadingSatire', 'notMisleadingOther',
                          'notMisleadingFactuallyCorrect',
                          'notMisleadingOutdatedButNotWhenWritten', 'notMisleadingClearlySatire',
                          'notMisleadingPersonalOpinion']
    
    # Find the column name with value 1
    for column in misleading_columns:
        if row[column] == 1:
            misleading_column = insert_spaces_capital(column)
            break

    # Fill in the prompt column
    prompt_text = f"Please explain the evidence behind your choice of labeling this tweet as {classification}, to help others who see this tweet understand why it is {misleading_column}"
    df_not_helpful_notes.at[index, 'prompt'] = prompt_text

In [ ]:
# Community Notes Data to KTO Trainer Format

# The data should be in the following format
'''kto_dataset_dict = {
    "prompt": [
        "Hey, hello",
        "How are you",
        "What is your name?",
        "What is your name?",
        "Which is the best programming language?",
        "Which is the best programming language?",
        "Which is the best programming language?",
    ],
    "completion": [
        "hi nice to meet you",
        "leave me alone",
        "I don't have a name",
        "My name is Mary",
        "Python",
        "C++",
        "Java",
    ],
    "label": [
        True,
        False,
        False,
        True,
        True,
        False,
        False,
    ]
}'''


import re
from langdetect import detect, LangDetectException

# Set the seed for reproducibility
np.random.seed(2002)

# Assuming df_helpful_notes and df_not_helpful_notes are your DataFrames
# Add a new column 'label' to indicate the label
df_helpful_notes['label'] = True
df_not_helpful_notes['label'] = False

# Combine the DataFrames
combined_df = pd.concat([df_helpful_notes, df_not_helpful_notes], ignore_index=True)

# Shuffle the indices
shuffled_indices = np.random.permutation(combined_df.index)
combined_df_shuffled = combined_df.iloc[shuffled_indices]


combined_df_shuffled = combined_df_shuffled.loc[:,["summary", "prompt", "label"]]

# Function to remove URLs from text
def remove_urls(text):
    return re.sub(r'http\S+', '', text)

# Filter out rows with missing or null values in the 'summary' column
combined_df_filtered = combined_df_shuffled.dropna(subset=['summary'])

# Remove URLs from the 'summary' column
combined_df_filtered['summary'] = combined_df_filtered['summary'].apply(lambda x: remove_urls(x) if isinstance(x, str) else None)

# Filter out rows where 'summary' text is not empty
combined_df_filtered = combined_df_filtered[combined_df_filtered['summary'].notnull() & (combined_df_filtered['summary'] != '')]

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException as e:
        print(f"Error detecting language: {e}")
        return None

# Add a new column 'language' to indicate the language of the summary
combined_df_filtered['language'] = combined_df_filtered['summary'].apply(detect_language)

# Filter out the rows where language detection was successful
combined_df_filtered = combined_df_filtered.dropna(subset=['language'])

# Filter out the English summaries
combined_df_english = combined_df_filtered[combined_df_filtered['language'] == 'en']

# Count the labels
combined_df_english['label'].value_counts()


In [ ]:
# Create kto_dataset_dict with only English summaries
kto_dataset_dict = {
    "prompt": combined_df_english['prompt'].tolist(),
    "completion": combined_df_english['summary'].tolist(),
    "label": combined_df_english['label'].tolist()
}

In [ ]:
## Optional if you want to export the data as csv

'''# Convert kto_dataset_dict to a DataFrame
df_kto_dataset_dict = pd.DataFrame(kto_dataset_dict)

# Export DataFrame to CSV file
df_kto_dataset_dict.to_csv('/Users/barisyazici/Desktop/community_noyes_kto.csv', index=False)'''